### Models

In [15]:
import torch
import numpy as np
torch.set_printoptions(edgeitems=2, precision=6, linewidth=75, sci_mode=False)

在本节中，我们将深入了解如何创建和使用模型。使用AutoModel类可以非常方便的从任意checkpoint实例化任何模型。
AutoModel类实际上是库中可用的各种模型的简单封装。其非常智能，可以根据checkpoint自动寻找到合适的模型架构，然后实例化该模型。
如果知道模型使用的具体类型，可以直接使用定义架构的类。让我们看看这是如何与BERT模型一起工作的。

#### Creating a Transformer

初始化BERT模型需要做的第一件事是加载一个配置对象:

In [16]:
from transformers import BertConfig, BertModel

# Building the config
config = BertConfig()

# Building the model from the config
model = BertModel(config)

config中包含许多用于构建模型的属性:

In [ ]:
config

这里的有些属性还未见过，其中：
- hidden_size属性定义了hidden_states向量的大小
- num_hidden_layers定义了Transformer模型的层数

#### Different loading methods

从默认配置中创建一个模型，用随机值初始化它:

In [18]:
from transformers import BertConfig, BertModel

# 这里加载的模型是随机的，且没有经过任何训练
config = BertConfig()
model = BertModel(config)

该模型在此状态下可以使用，但会输出乱码。首先模型需要被训练。我们可以根据手头的任务从头开始训练模型，但正如您在第一章中看到的那样，这将需要很长时间和大量数据，并且会对环境产生不可忽视的影响。为了避免不必要和重复的工作，应尽量共享和重用已经训练过的模型。
加载一个已经训练过的Transformer模型很简单——使用from_pretrained方法:

In [ ]:
from transformers import BertModel

model = BertModel.from_pretrained("bert-base-uncased")

正如前面看到的，可以用等价的AutoModel类替换BertModel。如果代码适用于一个checkpoint，那么它应该与另一个checkpoint无缝地工作。即使体系结构不同，只要checkpoint是为类似的任务(例如，情感分析任务)训练的，这也适用。

在上面的代码示例中，我们没有使用BertConfig，而是通过bert-base-case标识符加载了一个预训练的模型。这是一个模型checkpoint，是由BERT的作者自己训练的。

这个模型现在用checkpoint的所有权重初始化。它可以直接用于对训练任务的推断，也可以对新任务进行微调。通过使用预训练的权重进行训练，而不是从头开始，我们可以迅速取得良好的效果。

权重已被下载并缓存到缓存文件夹中(下次调用from_pretrained()方法时不会重新下载它们)，默认为~/.cache/huggingface/transformers。您可以通过设置HF_HOME环境变量来定制缓存文件夹。

用于加载模型的标识符可以是model Hub上任何模型的标识符，只要它与BERT体系结构兼容即可。可以在这里找到可用的BERT检查点的完整列表。

#### Saving methods

保存模型和加载模型一样简单——我们使用save_pretrained()方法，它类似于from_pretrained()方法:

In [20]:
model.save_pretrained("./output_models")

这将在磁盘上保存两个文件: config.json、pytorch_model.bin。

如果你看一下config.json文件，将看到构建模型架构所需的属性。该文件还包含一些元数据，例如checkpoint的起源位置以及上次保存检查点时使用的Transformers版本。

pytorch_model.bin文件被称为state dictionary，包含了模型的所有权重。这两个文件是成对出现的。config.json对于了解模型的体系结构是必要的，而模型权重是模型的参数。

#### Using a Transformer model for inference

现在知道了如何加载和保存模型，让我们尝试使用它进行一些预测。Transformer中的模型只能处理数字——tokenizer生成的数字。但在讨论tokenizer之前，让我们先探索一下模型接受哪些输入。

Tokenizers能将输入转换为适当的tensor，为了帮助理解发生了什么，我们将快速查看一下在将输入发送到模型之前必须做什么。

假设我们有几个序列:

In [21]:
sequences = ["Hello!", "Cool.", "Nice!"]

Tokenizer将这些转换为词汇表索引，通常称为input IDs。每个序列现在都是一个数字列表。结果输出为:

In [22]:
encoded_sequences = [
    [100, 7592, 999, 102],
    [100, 4658, 1012, 102],
    [100, 3835, 999, 102],
]

这是一个编码序列的列表:列表的列表。tensor只接受矩形尺寸(比如矩阵)。这个“数组”已经是矩形了，所以把它转换成张量很容易:

In [23]:
model_inputs = torch.tensor(encoded_sequences)
model_inputs, model_inputs.shape

(tensor([[ 100, 7592,  999,  102],
         [ 100, 4658, 1012,  102],
         [ 100, 3835,  999,  102]]),
 torch.Size([3, 4]))

#### Using the tensors as inputs to the model

在模型中使用张量是非常简单的，只需将模型称为带有inputs：

In [24]:
output = model(model_inputs)
output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.239509, -0.104296,  ..., -0.064994,  0.393062],
         [-1.263239, -0.368983,  ...,  0.024594, -0.397181],
         [-1.371740, -1.151259,  ..., -0.330527, -0.233956],
         [ 0.871134,  0.017735,  ..., -0.701856, -0.228727]],

        [[-0.100016,  0.110745,  ...,  0.293193,  0.514920],
         [ 0.425003, -0.341070,  ...,  0.046452,  0.079408],
         [-0.309742, -0.898449,  ...,  0.350212,  0.092437],
         [ 0.920230, -0.001883,  ..., -0.601311, -0.195279]],

        [[-0.056778,  0.142190,  ..., -0.078382,  0.336014],
         [ 0.230863, -0.221169,  ..., -0.129891, -0.435368],
         [-0.696269, -0.634549,  ..., -0.098256,  0.336197],
         [ 0.894659,  0.110280,  ..., -0.730504, -0.206675]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-0.777857, -0.148359,  ..., -0.485628,  0.850677],
        [-0.931797, -0.272293,  ..., -0.629579,  0.932008],
        [-0.755947, -

虽然模型接受许多不同的参数，但只有输入id是必需的。稍后我们将解释其他参数的作用以及何时需要它们，但首先我们需要仔细查看构建Transformer模型可以理解的输入的标记器。